In [1]:
x = {1:1}
x.pop(1)

1

In [3]:
from database import *
from pathlib import Path
workdb = WorkDB()

id_list = workdb.get_all_id()
id_list = sorted(id_list)
for id in id_list:
    a = workdb.get_all_epoch(id)
    print(id, a)

korean_abi_rec_v1 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,

In [2]:
import json

json.loads("""{'character': '지정', 'first': '자자', 'second': '이어', 'third': '으응'}""".replace("'", '"'))

{'character': '지정', 'first': '자자', 'second': '이어', 'third': '으응'}

In [12]:
import h5py
import numpy as np
from pathlib import Path

# Path("/home/dataset_cache.h5").unlink()
cache_file = "/home/dataset_cache.h5"
# HDF5 파일에 샘플 데이터 저장
def save_dict_to_hdf5(group, data):
    for key, value in data.items():
        if isinstance(value, dict):
            subgroup = group.create_group(key)
            save_dict_to_hdf5(subgroup, value)
        else:
            if isinstance(value, np.ndarray) and value.dtype.type is np.str_:
                dt = h5py.special_dtype(vlen=str)
                group.create_dataset(key, data=value.astype(dt))
            else:
                group.create_dataset(key, data=value)

def save_samples_to_hdf5(data_sample, sample_key, filename = cache_file):
    with h5py.File(filename, 'a') as f:
        if sample_key in f:
            return
        group = f.create_group(sample_key)
        save_dict_to_hdf5(group, data_sample)

# HDF5 파일에서 샘플 데이터 로드
def load_dict_from_hdf5(group):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = load_dict_from_hdf5(group[key])
        else:
            dataset = group[key]
            if dataset.shape == ():  # 스칼라 데이터셋
                value = dataset[()]
            else:
                value = dataset[:]
                if isinstance(value, np.ndarray) and value.dtype.type is np.bytes_:
                    value = value.astype(str)
            data[key] = value
    return data

def load_samples_from_hdf5(sample_key, filename = cache_file):
    with h5py.File(filename, 'r') as f:
        sample = load_dict_from_hdf5(f[sample_key])
    return sample

# 사용 예
data_sample1 = {
    'images': np.random.rand(64, 64, 3),
    'labels': np.array(0),
    'metadata': {
        'info': np.array(['sample1'], dtype='S'),
        'date': np.array(['2024-06-19'], dtype='S')
    }
}

data_sample2 = {
    'images': np.random.rand(64, 64, 3),
    'labels': np.array(1),
    'metadata': {
        'info': np.array(['sample2'], dtype='S'),
        'date': np.array(['2024-06-20'], dtype='S'),
        "a":{"a":1}
    }
}

# 샘플 데이터 저장
save_samples_to_hdf5(data_sample1, 'key_3')
save_samples_to_hdf5(data_sample2, 'key_4')

# 샘플 데이터 로드
loaded_sample1 = load_samples_from_hdf5('key_3')
loaded_sample2 = load_samples_from_hdf5('key_4')

# 로드된 데이터 출력
print("Loaded Sample 1:")
for key, value in loaded_sample1.items():
    print(f"  {key}: {value.shape if isinstance(value, np.ndarray) else value}")

print("\nLoaded Sample 2:")
for key, value in loaded_sample2.items():
    print(f"  {key}: {value.shape if isinstance(value, np.ndarray) else value}")



Loaded Sample 1:
  images: (64, 64, 3)
  labels: 0
  metadata: {'date': array(['2024-06-19'], dtype='<U10'), 'info': array(['sample1'], dtype='<U7')}

Loaded Sample 2:
  images: (64, 64, 3)
  labels: 1
  metadata: {'a': {'a': 1}, 'date': array(['2024-06-20'], dtype='<U10'), 'info': array(['sample2'], dtype='<U7')}


In [2]:
import os
import numpy as np

os.chdir("/home/code/PaddleOCR/ppocr")


from utils.dataset_cache import save_samples_to_hdf5, load_samples_from_hdf5



data_sample1 = {
    'images': np.random.rand(64, 64, 3),
    'labels': np.array(0),
    'metadata': {
        'info': np.array(['sample1'], dtype='S'),
        'date': np.array(['2024-06-19'], dtype='S')
    }
}

data_sample2 = {
    'images': np.random.rand(64, 64, 3),
    'labels': np.array(1),
    'metadata': {
        'info': np.array(['sample2'], dtype='S'),
        'date': np.array(['2024-06-20'], dtype='S'),
        "a":{"a":1}
    }
}


# 샘플 데이터 저장
save_samples_to_hdf5(data_sample1, 'key_1')
save_samples_to_hdf5(data_sample2, 'key_2')

# 샘플 데이터 로드
loaded_sample1 = load_samples_from_hdf5('key_1')
loaded_sample2 = load_samples_from_hdf5('key_2')

# 로드된 데이터 출력
print("Loaded Sample 1:")
for key, value in loaded_sample1.items():
    print(f"  {key}: {value.shape if isinstance(value, np.ndarray) else value}")

print("\nLoaded Sample 2:")
for key, value in loaded_sample2.items():
    print(f"  {key}: {value.shape if isinstance(value, np.ndarray) else value}")


Loaded Sample 1:
  images: (64, 64, 3)
  labels: 0
  metadata: {'date': array(['2024-06-19'], dtype='<U10'), 'info': array(['sample1'], dtype='<U7')}

Loaded Sample 2:
  images: (64, 64, 3)
  labels: 1
  metadata: {'a': {'a': 1}, 'date': array(['2024-06-20'], dtype='<U10'), 'info': array(['sample2'], dtype='<U7')}


In [42]:
from math import e
import h5py
import numpy as np
from pathlib import Path
from filelock import FileLock

import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'


# Path("/home/dataset_cache.h5").unlink()
cache_file = "/home/dataset_cache.h5"
lock = FileLock(f"{cache_file}.lock", timeout=10)

# HDF5 파일에 샘플 데이터 저장
def save_dict_to_hdf5(group, data):
    for key, value in data.items():
        if isinstance(value, dict):
            subgroup = group.create_group(key)
            save_dict_to_hdf5(subgroup, value)
        else:
            if isinstance(value, np.ndarray) and value.dtype.type is np.str_:
                dt = h5py.special_dtype(vlen=str)
                group.create_dataset(key, data=value.astype(dt))
            else:
                group.create_dataset(key, data=value)

def save_samples_to_hdf5(data_sample, sample_key, filename = cache_file):
    while True:
        try:
            with lock:
                with h5py.File(filename, 'a') as f:
                    if sample_key in f:
                        return
                    group = f.create_group(sample_key)
                    save_dict_to_hdf5(group, data_sample)
                    f.flush()
                    fd = f.id.get_vfd_handle()  # 파일 기술자 가져오기
                    os.fsync(fd)  # 운영 체제 파일 시스템 버퍼 플러시
                break
        except BlockingIOError as e:
            continue

# # HDF5 파일에서 샘플 데이터 로드
# def load_dict_from_hdf5(group):
#     data = {}
#     for key in group.keys():
#         if isinstance(group[key], h5py.Group):
#             data[key] = load_dict_from_hdf5(group[key])
#         else:
#             dataset = group[key]
#             if dataset.shape == ():  # 스칼라 데이터셋
#                 value = dataset[()]
#             else:
#                 value = dataset[:]
#                 if isinstance(value, np.ndarray) and value.dtype.type is np.bytes_:
#                     value = value.astype(str)
#             data[key] = value
#     return data

def load_dict_from_hdf5(group):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = load_dict_from_hdf5(group[key])
        else:
            dataset = group[key]
            if dataset.shape == ():  # 스칼라 데이터셋
                value = dataset[()]
                if isinstance(value, (bytes, np.bytes_)):
                    value = value.decode('utf-8')  # 바이트를 문자열로 디코딩
            else:
                value = dataset[:]
                if isinstance(value, np.ndarray) and value.dtype.type is np.bytes_:
                    value = value.astype(str)
                elif isinstance(value, np.ndarray) and value.dtype.type is np.object_:
                    value = np.array([item.decode('utf-8') if isinstance(item, (bytes, np.bytes_)) else item for item in value])
            data[key] = value
    return data


def load_samples_from_hdf5(sample_key, filename = cache_file):
    if not Path(filename).exists():
        return None
    with h5py.File(filename, 'r') as f:
        if sample_key in f:
            return load_dict_from_hdf5(f[sample_key])
        else:
            return None


a = 0
with lock:
    with h5py.File(cache_file, 'r') as f:
        for key in f["datasets"]["aihub_rec"].keys():
            try:
                a  += len(f["datasets"]["aihub_rec"][key].keys())
            except:
                
                pass
            
            
a
# load_samples_from_hdf5("./datasets/aihub_rec/718/717116.png").keys()
# load_samples_from_hdf5("./datasets/aihub_rec/718/717116.png")["origin_label"]


421805

In [9]:
pip install FileLock

Note: you may need to restart the kernel to use updated packages.


In [11]:
def chunk_list(data, num_chunks):
    avg = len(data) / float(num_chunks)
    chunks = []
    last = 0.0

    while last < len(data):
        chunks.append(data[int(last):int(last + avg)])
        last += avg

    return chunks

chunk_list(list(range(100)), 4)[0]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [1]:
cache_setting = {
    'rdcc_nbytes': 1024**3*20,  # 1GB
    'rdcc_nslots': int(1e6),  # 1백만 슬롯
    'rdcc_w0': 0.5  # 조기 쓰기 비율
}

from math import e
import h5py
import numpy as np
from pathlib import Path
from filelock import FileLock

import os
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'


# Path("/home/dataset_cache.h5").unlink()
cache_file = "/home/dataset_cache.h5"
lock = FileLock(f"{cache_file}.lock", timeout=10)

# HDF5 파일에 샘플 데이터 저장
def save_dict_to_hdf5(group, data):
    for key, value in data.items():
        if isinstance(value, dict):
            subgroup = group.create_group(key)
            save_dict_to_hdf5(subgroup, value)
        else:
            if isinstance(value, np.ndarray) and value.dtype.type is np.str_:
                dt = h5py.special_dtype(vlen=str)
                group.create_dataset(key, data=value.astype(dt))
            else:
                group.create_dataset(key, data=value)

def save_samples_to_hdf5(data_sample, sample_key, filename = cache_file):
    while True:
        try:
            with lock:
                with h5py.File(filename, 'w', **cache_setting) as f:
                    if sample_key in f:
                        return
                    group = f.create_group(sample_key)
                    save_dict_to_hdf5(group, data_sample)
                    f.flush()
                    fd = f.id.get_vfd_handle()  # 파일 기술자 가져오기
                    os.fsync(fd)  # 운영 체제 파일 시스템 버퍼 플러시
                break
        except BlockingIOError as e:
            continue

# # HDF5 파일에서 샘플 데이터 로드
# def load_dict_from_hdf5(group):
#     data = {}
#     for key in group.keys():
#         if isinstance(group[key], h5py.Group):
#             data[key] = load_dict_from_hdf5(group[key])
#         else:
#             dataset = group[key]
#             if dataset.shape == ():  # 스칼라 데이터셋
#                 value = dataset[()]
#             else:
#                 value = dataset[:]
#                 if isinstance(value, np.ndarray) and value.dtype.type is np.bytes_:
#                     value = value.astype(str)
#             data[key] = value
#     return data

def load_dict_from_hdf5(group):
    data = {}
    for key in group.keys():
        if isinstance(group[key], h5py.Group):
            data[key] = load_dict_from_hdf5(group[key])
        else:
            dataset = group[key]
            if dataset.shape == ():  # 스칼라 데이터셋
                value = dataset[()]
                if isinstance(value, (bytes, np.bytes_)):
                    value = value.decode('utf-8')  # 바이트를 문자열로 디코딩
            else:
                value = dataset[:]
                if isinstance(value, np.ndarray) and value.dtype.type is np.bytes_:
                    value = value.astype(str)
                elif isinstance(value, np.ndarray) and value.dtype.type is np.object_:
                    value = np.array([item.decode('utf-8') if isinstance(item, (bytes, np.bytes_)) else item for item in value])
            data[key] = value
    return data


def load_samples_from_hdf5(sample_key, filename = cache_file):
    if not Path(filename).exists():
        return None
    with h5py.File(filename, 'r') as f:
        if sample_key in f:
            while True:
                try:
                    return load_dict_from_hdf5(f[sample_key])
                except :
                    
                    continue
            
        else:
            return None


a = 0
with lock:
    with h5py.File(cache_file, 'r') as f:
        for key in f["datasets"]["aihub_rec"].keys():
            try:
                a  += len(f["datasets"]["aihub_rec"][key].keys())
            except BlockingIOError as e:
                pass
            
            
a
# load_samples_from_hdf5("./datasets/aihub_rec/718/717116.png").keys()
# load_samples_from_hdf5("./datasets/aihub_rec/718/717116.png")["origin_label"]


636448

In [5]:
import h5py

# h5py 라이브러리 버전 확인
print("h5py version:", h5py.__version__)

# 사용 중인 HDF5 라이브러리 버전 확인
print("HDF5 version:", h5py.version.hdf5_version)


h5py version: 3.8.0
HDF5 version: 1.12.2


In [9]:
x = {1, 2, 3}
y= {3, 4, 5}
z = {1, 3, 5}
a = [x, y, z]


import itertools
set(itertools.chain.from_iterable(a))

{1, 2, 3, 4, 5}

In [10]:
a = "sdf.pdparams"
a[:-8]

'sdf.'